In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px



url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
USCONFIRMED = pd.read_csv(url)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
USDEATHS = pd.read_csv(url)
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
CONFIRMED = pd.read_csv(url)

In [ ]:
# calculate mortality rate of each state in the US
USCONFIRMED = USCONFIRMED.groupby('Province_State',axis = 0).sum()
USDEATHS = USDEATHS.groupby('Province_State',axis = 0).sum()
Ratio = USDEATHS.iloc[:,-1]/USCONFIRMED.iloc[:,-1]
Ratio[Ratio.isna()] = 0
RatioIndex = Ratio.sort_values(axis = 0, ascending= False)

In [ ]:
# define a function to draw the subplot for visualizing
def fonction(x): 
  COUNTRYCONFIRMED = CONFIRMED.iloc[:,[1,-1]]
  sum_COUNTRYCONFIRMED = COUNTRYCONFIRMED.groupby(COUNTRYCONFIRMED.columns[0],axis = 0).sum()
  COUNTRYINDEX = sum_COUNTRYCONFIRMED[sum_COUNTRYCONFIRMED.columns[-1]].sort_values(axis = 0, ascending= False).head(x)
  RatioIndexTmp = RatioIndex.head(x)

  fig = make_subplots(
    rows=2, cols=2,
    column_widths=[0.5, 0.5],
    row_heights=[.5, .5],
    specs=[[{'type': 'Scatter', 'colspan': 2}, None],
           [{'type': 'bar'}, {'type': 'Pie'}]],
    subplot_titles=('Confirmed Cases Trends for Top '+str(x)+' Affected Countires','Confirmed Cases Updated for Top '+str(x)+' Affected Countires', 
                    'Top '+str(x)+' Covid 19 Mortality Rate in the US, by state')
    )
 
  for i in np.arange(0,x):
    y = CONFIRMED[CONFIRMED['Country/Region']== COUNTRYINDEX.index[i]]
    sum_y = y.sum(axis=0)
    sum_y = pd.DataFrame(sum_y)
    sum_y = sum_y.iloc[4:]
    sum_y.rename(columns={0: 'ConfirmedCases'}, inplace=True)
    a = len(sum_y)
    fig.add_trace(go.Scatter(x=sum_y.index, y=sum_y['ConfirmedCases'],name = COUNTRYINDEX.index[i], mode='lines'), 
    row = 1, col = 1)
  # Add second subplot(bar plot)
  fig.add_trace(
    go.Bar(x = COUNTRYINDEX.index, y = COUNTRYINDEX, showlegend= False),
    row=2, col=1
  )
  # Add third subplot(pie chart)
  fig.add_trace(go.Pie(labels = RatioIndexTmp.index,values = RatioIndexTmp),
                row = 2, col = 2
  )
  # Rotate x-axis labels
  fig.update_xaxes(tickangle=-45)
  # Set theme, margin and font size in layout
  fig.update_layout(
    template='plotly_dark',
    margin=dict(r=10, t=25, b=40, l=60),
    font = dict(size = 10)
  )
  fig.show()

In [ ]:
import pandas as pd
#download confirmed cases data from JHU dashboard
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
df_confirmed = pd.read_csv(url)

#download death cases data from JHU dashboard
url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
df_death = pd.read_csv(url)

#top ten impacted countries 
df_tmp_subset = df_confirmed.drop(['Province/State','Lat','Long'],axis=1)
df_tmp_subset = pd.DataFrame(df_tmp_subset.groupby(['Country/Region'],as_index=False).sum())
df_tmp_subset = df_tmp_subset.sort_values(by=df_confirmed.columns[len(df_confirmed.columns)-1], ascending=False)
df_topten_countries = df_tmp_subset[0:10]

#death count in the top ten most impacted countries 
df_tmp_subset_d = df_death.drop(['Province/State','Lat','Long'],axis=1)
df_tmp_subset_d = pd.DataFrame(df_tmp_subset_d.groupby(['Country/Region'],as_index=False).sum())
df_tmp_subset_d = df_tmp_subset_d[df_tmp_subset_d['Country/Region'].isin(df_topten_countries['Country/Region'].values.flatten().tolist())]
df_topten_death = df_tmp_subset_d.sort_values(by=df_tmp_subset_d.columns[len(df_tmp_subset_d.columns)-1], ascending=False)

In [ ]:
fonction(10)

In [ ]:
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

x_axis = df_topten_countries.columns
fig = go.Figure()
annotations = []

for i in range(0,10,1):
  y = df_topten_countries.iloc[i,1:].values.flatten().tolist()
  fig.add_trace(go.Scatter(x=x_axis[1:], y=y,
                    mode='lines+markers',
                    name=df_topten_countries.iloc[i,0]))

# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Confirmed Case Trend of Top Ten Impacted Countries',
                              font=dict(family='Arial',
                                        size=18,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
fig.update_layout(annotations=annotations)
fig.show()

# Add table data
table_data = [['Country', 'Confirmed', 'Deaths'],
              [df_topten_countries.iloc[0,0], df_topten_countries.iloc[0,-1], df_topten_death.iloc[0,-1]],
              [df_topten_countries.iloc[1,0], df_topten_countries.iloc[1,-1], df_topten_death.iloc[1,-1]],
              [df_topten_countries.iloc[2,0], df_topten_countries.iloc[2,-1], df_topten_death.iloc[2,-1]],
              [df_topten_countries.iloc[3,0], df_topten_countries.iloc[3,-1], df_topten_death.iloc[3,-1]],
              [df_topten_countries.iloc[4,0], df_topten_countries.iloc[4,-1], df_topten_death.iloc[4,-1]],
              [df_topten_countries.iloc[5,0], df_topten_countries.iloc[5,-1], df_topten_death.iloc[5,-1]],
              [df_topten_countries.iloc[6,0], df_topten_countries.iloc[6,-1], df_topten_death.iloc[6,-1]],
              [df_topten_countries.iloc[7,0], df_topten_countries.iloc[7,-1], df_topten_death.iloc[7,-1]],
              [df_topten_countries.iloc[8,0], df_topten_countries.iloc[8,-1], df_topten_death.iloc[8,-1]],
              [df_topten_countries.iloc[9,0], df_topten_countries.iloc[9,-1], df_topten_death.iloc[9,-1]]]

# Initialize a figure with ff.create_table(table_data)
fig = ff.create_table(table_data, height_constant=40)

# Add graph data
x_axis_bar = df_topten_countries.iloc[:,0].values.flatten().tolist()
conf_count = df_topten_countries.iloc[:,-1].values.flatten().tolist()
death_count = df_topten_death.iloc[:,-1].values.flatten().tolist()

# Make traces for graph
trace1 = go.Bar(x=x_axis_bar, y=conf_count, xaxis='x2', yaxis='y2',
                marker=dict(color='#0099ff'),
                name='Confirmed Cases')
trace2 = go.Bar(x=x_axis_bar, y=death_count, xaxis='x2', yaxis='y2',
                marker=dict(color='#ff001a'),
                name='Death Cases')

# Add trace data to figure
fig.add_traces([trace1, trace2])

# initialize xaxis2 and yaxis2
fig['layout']['xaxis2'] = {}
fig['layout']['yaxis2'] = {}

# Edit layout for subplots
fig.layout.yaxis.update({'domain': [0, .45]})
fig.layout.yaxis2.update({'domain': [.6, 1]})

# The graph's yaxis2 MUST BE anchored to the graph's xaxis2 and vice versa
fig.layout.yaxis2.update({'anchor': 'x2'})
fig.layout.xaxis2.update({'anchor': 'y2'})
fig.layout.yaxis2.update({'title': 'Case Count'})

# Update the margins to add a title and see graph x-labels.
fig.layout.margin.update({'t':75, 'l':50})
fig.layout.update({'title': 'COVID19 Confirmed and Death Case count'})

# Update the height because adding a graph vertically will interact with
# the plot height calculated for the table
fig.layout.update({'height':600})

# Plot!
fig.show()